# Notebook 4: Compute d3 for testing facilities and residence centroids and visualize routes

## Introduction to Noteboook 4

Note: Use content from Advanced 1

## Data sources

In [1]:
import pandas as pd, geopandas as gpd, folium

pd.options.display.float_format = '{:.10f}'.format

### Testing Facilities, Parish, Residence Centroids

In [2]:
filtered_testing_sites_4326_gdf = gpd.read_file('data/filtered_testing_sites_4326_gdf.gpkg')
parish_gdf = gpd.read_file('data/parish_gdf.gpkg')
residential_centroids_4326_gdf = gpd.read_file('data/residential_centroids_4326_gdf.gpkg')

In [3]:
residential_centroids_4326_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 213 entries, 0 to 212
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   element_type  213 non-null    object  
 1   osmid         213 non-null    int64   
 2   building      213 non-null    object  
 3   lat           213 non-null    float64 
 4   lon           213 non-null    float64 
 5   prj_lat       213 non-null    float64 
 6   prj_lon       213 non-null    float64 
 7   r_node        213 non-null    int64   
 8   r_node_lat    213 non-null    float64 
 9   r_node_lon    213 non-null    float64 
 10  r_node_y      213 non-null    float64 
 11  r_node_x      213 non-null    float64 
 12  d1            213 non-null    float64 
 13  d1_euc        213 non-null    float64 
 14  geometry      213 non-null    geometry
dtypes: float64(10), geometry(1), int64(2), object(2)
memory usage: 25.1+ KB


In [4]:
residential_centroids_4326_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [5]:
residential_centroids_4326_gdf

,element_type,osmid,building,lat,lon,prj_lat,prj_lon,r_node,r_node_lat,r_node_lon,r_node_y,r_node_x,d1,d1_euc,geometry
0,way,140823106,residential,0.3050501833,32.6082495446,33718.0074585697,456408.2543575270,6227266298,0.3045983000,32.6083010000,33668.0593837221,456413.9782893857,50.2749796310,50.2749796310,POINT (32.60825 0.30505)
1,way,140823115,dormitory,0.3053414841,32.6082456333,33750.2057395246,456407.8202964029,579993314,0.3055411000,32.6084585000,33772.2689535378,456431.5080539904,32.3712105445,32.3712105445,POINT (32.60825 0.30534)
2,way,140826566,duplex,0.3045664193,32.6084960943,33664.5347349995,456435.6874821403,6227266298,0.3045983000,32.6083010000,33668.0593837221,456413.9782893857,21.9934580881,21.9934580881,POINT (32.60850 0.30457)
3,way,140826574,residential,0.3045408000,32.6083606500,33661.7035135770,456420.6156729474,6227266298,0.3045983000,32.6083010000,33668.0593837221,456413.9782893857,9.1897739823,9.1897739823,POINT (32.60836 0.30454)
4,way,142109314,residential,0.3046784338,32.6080187078,33676.9179346887,456382.5662585886,6227266299,0.3045855000,32.6082708000,33666.6446875843,456410.6177021883,29.8734513255,29.8734513255,POINT (32.60802 0.30468)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,way,566748730,house,0.3086906479,32.6082211451,34120.3978818522,456405.1089029149,579993370,0.3084251000,32.6083627000,34091.0455836289,456420.8594871131,33.3112340446,33.3112340446,POINT (32.60822 0.30869)
209,way,566748732,house,0.3088099936,32.6081564965,34133.5897534508,456397.9155612630,1460122475,0.3091566000,32.6080193000,34171.9016473888,456382.6502965369,41.2411144888,41.2411144888,POINT (32.60816 0.30881)
210,way,566748733,house,0.3087290884,32.6080501563,34124.6475147437,456386.0821231636,579993372,0.3083446000,32.6082158000,34082.1483045059,456404.5127182511,46.3235329624,46.3235329624,POINT (32.60805 0.30873)
211,way,569163822,house,0.3090996240,32.6081822843,34165.6032674131,456400.7863013543,6226990677,0.3092163000,32.6081950000,34178.4997268183,456402.2017287260,12.9739007177,12.9739007177,POINT (32.60818 0.30910)


In [6]:
filtered_testing_sites_4326_gdf.shape

(16, 17)

In [7]:
filtered_testing_sites_4326_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   FACILITY     16 non-null     object  
 1   CITY         16 non-null     object  
 2   ADDRESS      16 non-null     object  
 3   LAT          16 non-null     float64 
 4   LON          16 non-null     float64 
 5   COORDINATES  16 non-null     object  
 6   NOTES        2 non-null      object  
 7   PRJ_LAT      16 non-null     float64 
 8   PRJ_LON      16 non-null     float64 
 9   t_node       16 non-null     int64   
 10  t_node_lat   16 non-null     float64 
 11  t_node_lon   16 non-null     float64 
 12  t_node_y     16 non-null     float64 
 13  t_node_x     16 non-null     float64 
 14  d2           16 non-null     float64 
 15  d2_euc       16 non-null     float64 
 16  geometry     16 non-null     geometry
dtypes: float64(10), geometry(1), int64(1), object(5)
memory usage: 2.2+

In [8]:
filtered_testing_sites_4326_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [9]:
filtered_testing_sites_4326_gdf

,FACILITY,CITY,ADDRESS,LAT,LON,COORDINATES,NOTES,PRJ_LAT,PRJ_LON,t_node,t_node_lat,t_node_lon,t_node_y,t_node_x,d2,d2_euc,geometry
0,Central Public Health Laboratory,Kampala,"7/11, Plot 113 Buganda Rd, Kampala, Uganda",0.3312460000,32.5761710000,"0.331245631028126, 32.57617147103373",None,36613.6482006860,452838.7936172243,3799704477,0.3311248000,32.5760535000,36600.2521446763,452825.7180929286,18.7196060915,18.7196060915,POINT (32.57617 0.33125)
1,Infectious Disease Institute Laboratory,Kampala,"P.O.Box 22418, Kampala, Uganda",0.3391550000,32.5761190000,"0.3391550027171229, 32.57611913788221",None,37487.8549663987,452833.0450933628,7401202859,0.3390506000,32.5759309000,37476.3162241753,452812.1135546908,23.9012945941,23.9012945941,POINT (32.57612 0.33916)
2,Makerere University,Kampala,"University Rd, Kampala, Uganda",0.3337660000,32.5675150000,"0.33376643025242, 32.56751532874441",None,36892.2336127046,451875.5986544515,2297820937,0.3338064000,32.5674293000,36896.6995755768,451866.0624737106,10.5301266611,10.5301266611,POINT (32.56752 0.33377)
3,Mild May Laboratory,Kampala,"6HG2+QJH, Kampala, Uganda",0.2272610000,32.5514940000,"0.22726149143899727, 32.551493611083714",None,25119.9449333336,450092.3882960156,2614743709,0.2274452000,32.5514859000,25140.3052270359,450091.4875805033,20.3802072629,20.3802072629,POINT (32.55149 0.22726)
4,Joint Clinical Research Center (JCRC),Kampala,"P.o.Box 10005, Kampala, Uganda",0.2471060000,32.5615450000,"0.24710642516379605, 32.56154518525522",None,27313.4443953668,451210.9074285642,7062105534,0.2489300000,32.5610400000,27515.0590496562,451154.7191193702,209.2978617050,209.2978617050,POINT (32.56155 0.24711)
5,MBN Laboroatory,Kampala,"Plot 28 Nakasero Rd, Kampala, Uganda",0.3244010000,32.5768040000,"0.3244006304886406, 32.57680365762819",None,35857.0460002950,452909.1994618282,6880975575,0.3244436000,32.5767374000,35861.7550210679,452901.7886607080,8.7803672978,8.7803672978,POINT (32.57680 0.32440)
6,Medipal International Hospital,Kampala,"John Babiha (Acacia) Ave, Kampala, Uganda",0.3267710000,32.5876990000,"0.32677070175063294, 32.58769862875094",None,36118.9590743176,454121.5640353452,8193448456,0.3265384000,32.5878109000,36093.2485964963,454134.0147725081,28.5665805742,28.5665805742,POINT (32.58770 0.32677)
7,Test and Fly Laboratory,Kampala,"Yusuf Lule Road, Kampala, Uganda",0.3280000000,32.5833240000,"0.3279995303971809, 32.58332419987116",None,36254.8242478438,453634.7367577547,7238684605,0.3280009000,32.5832163000,36254.9242265742,453622.7523243336,11.9848504443,11.9848504443,POINT (32.58332 0.32800)
8,Uganda Cancer Institute,Kampala,"Upper Mulago Hill Rd, Kampala, Uganda",0.3415660000,32.5779390000,"0.34156560138915515, 32.577938699857306",None,37754.3414306474,453035.5792187499,6232768975,0.3417020000,32.5779317000,37769.3739685455,453034.7675639616,15.0544338702,15.0544338702,POINT (32.57794 0.34157)
9,IOM Laboratory,Kampala,"Plot 6A Bukoto Crescent, Naguru, Kampala 11431...",0.3419140000,32.6050250000,"0.3419138946476636, 32.6050250521126",None,37792.6787080598,456049.5983998349,560476404,0.3423478000,32.6052258000,37840.6268860078,456071.9445241932,52.8996884903,52.8996884903,POINT (32.60502 0.34191)


### Projectted OSMNx graph for Kampala

In [10]:
import osmnx as ox, csv

with open('overpass-api.csv', mode='r') as infile:
    reader = csv.reader(infile)
    overpass_api = {rows[0]:rows[1] for rows in reader}

ox.config(
    log_console=True, 
    use_cache=True, 
    log_file=True,
    overpass_endpoint=overpass_api['main']
)

In [11]:
%%time
G_proj = ox.load_graphml('data/g_projected.graphml')

CPU times: user 47.7 s, sys: 4.66 s, total: 52.4 s
Wall time: 52.3 s


In [12]:
G_proj.graph['crs']

'+proj=utm +zone=36 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs'

## Create residence-test facility pair DF and analysis columns

### Create a Pandas DataFrame (DF) to store results of d1, d2, and d3 computation

In [13]:
%%time
import networkx as nx, os
from tqdm import tqdm, notebook
from shapely.geometry import LineString


dict_list = []
df_list = []

# tqdm parameters
total_rows=5 #residential_centroids_4326_gdf.shape[0]

for r_index, r_row in tqdm(residential_centroids_4326_gdf.sample(n=total_rows, random_state=1).iterrows(), total=total_rows, desc='Residence Loop'):
    
    # for each sampled residence centroid, compute d3 and other parameters
    # create a dictionary for each pair and create a temporary DF for each dictionary
    dict_list = []
    for t_index, t_row in filtered_testing_sites_4326_gdf.iterrows():
        
        # 1. assemble the paired record using the nested loop
        r_node = r_row['r_node']
        r_node_lat = r_row['r_node_lat']
        r_node_lon = r_row['r_node_lon']
        t_node = t_row['t_node']
        t_node_lat = t_row['t_node_lat']
        t_node_lon = t_row['t_node_lon']
        d1 = r_row['d1']
        d2 = t_row['d2']
        
        # List if OSMNx nodes for shortest path
        path_node_list = ox.distance.shortest_path(G_proj, r_node, t_node, weight='speed', cpus=2)

        ## d3 method 1 (sum of euclidean distance of path edges)
        # simplified explanation: 
        # 1. node to node distance: osmnx.distance.euclidean_dist_vec(source_lat, source_lon, target_lat, target_lon, distance)
        # 2. dist_list: list of node to node distances generated through Python list comprehension (for loop)
        # 3. d3_path_sum: sum of distances in dist_list
        dist_list = [ ox.distance.euclidean_dist_vec(G_proj.nodes[path_node_list[i]]['y'], G_proj.nodes[path_node_list[i]]['x'], \
                                                  G_proj.nodes[path_node_list[i+1]]['y'], G_proj.nodes[path_node_list[i+1]]['x']) for i in range(len(path_node_list)-1) ]
        d3_path_sum = sum(dist_list)

        ## d3 method 2 (Shapely LineString Length)
        coords_list = [(G_proj.nodes[node]['x'], G_proj.nodes[node]['y']) for node in path_node_list ]
        path_line = LineString(coords_list)
        d3_shapely = path_line.length

        ## d3 method 3 (euclidean distance between residence centroid and test facility coordinates
        d3_euc = ox.distance.euclidean_dist_vec(r_row['prj_lat'], r_row['prj_lon'], t_row['PRJ_LAT'], t_row['PRJ_LON'])
                        
        d_total = d1 + d2 + d3_path_sum
        # assemble record dictionary
        record_dict = {
            'r_node': r_node,
            'r_node_lat': r_node_lat,
            'r_node_lon': r_node_lon,
            't_node': t_node,
            't_node_lat': t_node_lat,
            't_node_lon': t_node_lon,
            'd3_euc': d3_euc,
            'd3_shapely': d3_shapely,
            'path_node_list': path_node_list,
            'path_distances': dist_list,
            'd3_path_sum': d3_path_sum,
            'd_total': d_total
        }
        #print(record_dict)
        
        # append record dict to dict_list (each record is a future DF row, each dict_list is future temp DF)
        dict_list.append(record_dict)
    
    # 2. Create a temporary DF to store list of record dictionaries (these become DF rows)
    # Sort values by the d3_path_sum column
    # Group by r_node (remember each temp DF contains records for only one r_node)
    # Retain the first record (which would have the smallest value of d_total)
    df = pd.DataFrame(dict_list).sort_values(by=['d_total']).groupby(['r_node'], as_index=False).first()
    # append this temp DF to a DF list
    df_list.append(df)

# 3. Concatenate (combine) all the temp DFs in the DF list to a single DF
# Reset the index of the DF and drop the original indices of the temp DFs
paired_df = pd.concat(df_list).reset_index(drop=True)

# 4. Convert the path_node_list column to type string
paired_df['path_node_list'] = paired_df['path_node_list'].astype(str)

paired_df

Residence Loop: 100%|██████████| 5/5 [01:17<00:00, 15.49s/it]

CPU times: user 1min 16s, sys: 1.29 s, total: 1min 17s
Wall time: 1min 17s


,r_node,r_node_lat,r_node_lon,t_node,t_node_lat,t_node_lon,d3_euc,d3_shapely,path_node_list,path_distances,d3_path_sum,d_total
0,475033488,0.3109017000,32.6098430000,7401065633,0.3076673000,32.6249390000,1694.6835003641,2617.9292724860,"[475033488, 1583799847, 475033491, 3481960627,...","[34.857797441936704, 18.888149019910713, 9.026...",2617.9292724860,2724.9654919912
1,579992776,0.3087764000,32.6061156000,7401065633,0.3076673000,32.6249390000,2025.1583863109,3400.9981796060,"[579992776, 6226990662, 6226990663, 6226990645...","[13.7069580255798, 11.075395892702314, 6.39105...",3400.9981796060,3537.4031100213
2,579993404,0.3097587000,32.6091322000,7401065633,0.3076673000,32.6249390000,1745.8336543244,2921.4964514234,"[579993404, 475033943, 579993409, 475033948, 4...","[25.50617968399029, 51.7302532559078, 49.87416...",2921.4964514234,3026.7372672074
3,1583799787,0.3100471000,32.6100813000,7401065633,0.3076673000,32.6249390000,1640.4063742119,2677.1014087277,"[1583799787, 6227589978, 1583800118, 158380034...","[9.721294101490457, 7.796506912348756, 12.5149...",2677.1014087277,2777.0766747860
4,8144996943,0.3033524000,32.6059574000,8003448502,0.3007283000,32.5890897000,1892.8943434713,2774.3196406058,"[8144996943, 8144996944, 579992620, 579992609,...","[46.50197410980276, 51.71860469243541, 56.8267...",2774.3196406058,2800.6361888393


## Map Paired Residence Centroids and Test Facilities and Node-to-Node Routes

In [14]:
%%time
import folium, json
from folium import plugins

map1 = filtered_testing_sites_4326_gdf.explore(marker_kwds=dict(radius=5))

# Tile Layer (can add more, these become radio buttons on Layer Control)
folium.TileLayer('cartodbpositron').add_to(map1)

# Feature groups become checkboxes in Layer Control Widget
fg1=folium.FeatureGroup(name='Residences', show=True)
fg2=folium.FeatureGroup(name='Residence Nodes', show=True)
fg3=folium.FeatureGroup(name='Residence to Node', show=True)

# Residences, Nodes and Residence-to-Node Paths
for index,row in residential_centroids_4326_gdf.iterrows():

    folium.CircleMarker(
                    location=[row['lat'],row['lon']], \
                    radius=4, \
                    color='black', \
                    weight=1, \
                    fill=True, \
                    fill_color='red', \
                    fill_opacity=1).add_to(fg1)


    folium.CircleMarker(
                    location=[row['r_node_lat'],row['r_node_lon']], \
                    radius=4, \
                    color='black', \
                    weight=1, \
                    fill=True, \
                    fill_color='yellow', \
                    fill_opacity=1).add_to(fg2)
    
    r_line_points = ((row['lat'],row['lon']),(row['r_node_lat'],row['r_node_lon']))
    folium.PolyLine(r_line_points,
                    color='gray',
                    weight=1,
                    opacity=0.8
                   ).add_to(fg3)

fg1.add_to(map1)
fg2.add_to(map1)
fg3.add_to(map1)

# Residences, Nodes and Residence-to-Node Paths
fg4=folium.FeatureGroup(name='Testing Sites', show=True)
fg5=folium.FeatureGroup(name='Testing Site Nodes', show=True)
fg6=folium.FeatureGroup(name='Testing Site to Node', show=True)

for index,row in filtered_testing_sites_4326_gdf.iterrows():

    folium.CircleMarker(
                    location=[row['LAT'],row['LON']], \
                    radius=4, \
                    color='black', \
                    weight=1, \
                    fill=True, \
                    fill_color='blue', \
                    fill_opacity=1).add_to(fg4)


    folium.CircleMarker(
                    location=[row['t_node_lat'],row['t_node_lon']], \
                    radius=4, \
                    color='black', \
                    weight=1, \
                    fill=True, \
                    fill_color='yellow', \
                    fill_opacity=1).add_to(fg5)
    
    t_line_points = ((row['LAT'],row['LON']),(row['t_node_lat'],row['t_node_lon']))
    folium.PolyLine(t_line_points,
                    color='gray',
                    weight=1,
                    opacity=0.8
                   ).add_to(fg6)
fg4.add_to(map1)
fg5.add_to(map1)
fg6.add_to(map1)

# Node to Node Routes
fg7=folium.FeatureGroup(name='Node to Node Routes', show=True)
for index, row in paired_df.sample(n=5,random_state=1).iterrows():
    path_string = row['path_node_list']
    path_node_list = json.loads(path_string)
    edge_list = []
    for i in range(len(path_node_list)-1):
        edge_pair = ((G_proj.nodes[path_node_list[i]]['lat'], G_proj.nodes[path_node_list[i]]['lon']), \
                     (G_proj.nodes[path_node_list[i+1]]['lat'], G_proj.nodes[path_node_list[i+1]]['lon']))
        edge_list.append(edge_pair)
        popup = folium.Popup(str(row['d_total']/1000)+' km')
        folium.PolyLine(edge_list,
                        color='red',
                        weight=2,
                        opacity=0.8,
                        popup=popup
                       ).add_to(fg7)
fg7.add_to(map1)

# GeoJSON parish boundary with style function
fg8=folium.FeatureGroup(name='Parish Boundary', show=True)
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0, 
                            'weight': 3}
geojson = folium.GeoJson(
    data=parish_gdf['geometry'], 
    name="geojson",
    style_function=style_function
)
geojson.add_child(folium.Popup('Parish Boundary'))
geojson.add_to(fg8)
fg8.add_to(map1)

# Layer Control
folium.LayerControl(position='topright', collapsed=True, autoZIndex=True).add_to(map1)

# Full Screen button
plugins.Fullscreen(
    position='topright',
    title='Expand me',
    title_cancel='Exit me',
    force_separate_button=True
).add_to(map1)

map1

CPU times: user 1.45 s, sys: 59.7 ms, total: 1.51 s
Wall time: 1.5 s


## Housekeeping

In [15]:
paired_df.to_pickle('data/paired_df.pickle')